# Elo world 

In this kernel, I build a LGBM model that aggregates the `new_merchant_transactions.csv` and `historical_transactions.csv` tables to the main train table. New features are built by successive grouping on`card_id` and `month_lag`, in order to recover some information from the time serie.

During the competition, I took into account the enlightments provided by others kernels, and included a few features that appeared to be important. In particular, I closely looked at the following kernels (ordered by release time):
1.  [You're Going to Want More Categories [LB 3.737] by Peter Hurford](https://www.kaggle.com/peterhurford/you-re-going-to-want-more-categories-lb-3-737)
2. [EloDA with Feature Engineering and Stacking by Bojan Tunguz](https://www.kaggle.com/tunguz/eloda-with-feature-engineering-and-stacking)
3. [A Closer Look at Date Variables by Robin Denz](https://www.kaggle.com/denzo123/a-closer-look-at-date-variables)
4. [LGB + FE (LB 3.707) by Konrad Banachewicz](https://www.kaggle.com/konradb/lgb-fe-lb-3-707)
5. [My first kernel (3.699) by Chau Ngoc Huynh](https://www.kaggle.com/chauhuynh/my-first-kernel-3-699/)

## Notebook  Content
1. [Loading the data](#1)
1. [Feature engineering](#2)
1. [Training the model](#3)
1. [Feature importance](#4)
1. [Submission](#5)

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

<a id="1"></a> <br>
## 1. Loading the data

First, we load the `new_merchant_transactions.csv` and `historical_transactions.csv`. In practice, these two files contain the same variables and the difference between the two tables only concern the position with respect to a reference date.  Also, booleans features are made numeric:

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
new_transactions = pd.read_csv('../../../input/new_merchant_transactions.csv',parse_dates=['purchase_date'])

historical_transactions = pd.read_csv('../../../input/historical_transactions.csv',parse_dates=['purchase_date'])

def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

historical_transactions = binarize(historical_transactions)
new_transactions = binarize(new_transactions)

We then load the main files, formatting the dates and extracting the target:

In [4]:
def read_data(input_file):
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (datetime.date(2018, 2, 1) - df['first_active_month'].dt.date).dt.days
    return df
#_________________________________________
train = read_data('../../../input/train.csv')
test = read_data('../../../input/test.csv')

target = train['target']
#del train['target']

<a id="2"></a> <br>
## Feature engineering
Following [Chau Ngoc Huynh's kernel](https://www.kaggle.com/chauhuynh/my-first-kernel-3-699/), I add the following features:

In [5]:
historical_transactions['month_diff'] = ((datetime.datetime.today() - historical_transactions['purchase_date']).dt.days)//30
historical_transactions['month_diff'] += historical_transactions['month_lag']

new_transactions['month_diff'] = ((datetime.datetime.today() - new_transactions['purchase_date']).dt.days)//30
new_transactions['month_diff'] += new_transactions['month_lag']

In [6]:
train['outliers'] =[1 if i < -30 else 0 for i in target] 

In [7]:
historical_transactions[:5]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,month_diff
0,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,11
1,1,C_ID_4e6213e9bc,88,0,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,11
2,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,12
3,1,C_ID_4e6213e9bc,88,0,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,12
4,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,12


In [8]:
historical_transactions = pd.get_dummies(historical_transactions, columns=['category_2', 'category_3'])
new_transactions = pd.get_dummies(new_transactions, columns=['category_2', 'category_3'])

historical_transactions = reduce_mem_usage(historical_transactions)
new_transactions = reduce_mem_usage(new_transactions)

agg_fun = {'authorized_flag': ['mean']}
auth_mean = historical_transactions.groupby(['card_id']).agg(agg_fun)
auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]
auth_mean.reset_index(inplace=True)

authorized_transactions = historical_transactions[historical_transactions['authorized_flag'] == 1]
historical_transactions = historical_transactions[historical_transactions['authorized_flag'] == 0]

Mem. usage decreased to 1332.66 Mb (57.1% reduction)
Mem. usage decreased to 86.12 Mb (58.9% reduction)


First, following [Robin Denz](https://www.kaggle.com/denzo123/a-closer-look-at-date-variables) analysis, I define a few dates features:

In [9]:
historical_transactions['purchase_month'] = historical_transactions['purchase_date'].dt.month
authorized_transactions['purchase_month'] = authorized_transactions['purchase_date'].dt.month
new_transactions['purchase_month'] = new_transactions['purchase_date'].dt.month

historical_transactions['purchase_day'] = historical_transactions['purchase_date'].dt.day
authorized_transactions['purchase_day'] = authorized_transactions['purchase_date'].dt.day
new_transactions['purchase_day'] = new_transactions['purchase_date'].dt.day

historical_transactions['purchase_hour'] = historical_transactions['purchase_date'].dt.hour
authorized_transactions['purchase_hour'] = authorized_transactions['purchase_date'].dt.hour
new_transactions['purchase_hour'] = new_transactions['purchase_date'].dt.hour

Then I define two functions that aggregate the info contained in these two tables. The first function aggregates the function by grouping on `card_id`:

In [10]:
def aggregate_transactions(history):
    
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).\
                                      astype(np.int64) * 1e-9
    
    agg_func = {
    'category_1': ['sum', 'mean'],
    'category_2_1.0': ['mean'],
    'category_2_2.0': ['mean'],
    'category_2_3.0': ['mean'],
    'category_2_4.0': ['mean'],
    'category_2_5.0': ['mean'],
    'category_3_A': ['mean'],
    'category_3_B': ['mean'],
    'category_3_C': ['mean'],
    'merchant_id': ['nunique'],
    'merchant_category_id': ['nunique'],
    'state_id': ['nunique'],
    'city_id': ['nunique'],
    'subsector_id': ['nunique'],
    'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
    'installments': ['sum', 'mean', 'max', 'min', 'std'],
    'purchase_month_sin': ['mean', 'max', 'min', 'std'],
    'purchase_month_cos': ['mean', 'max', 'min', 'std'],
    'purchase_day_sin': ['mean', 'max', 'min', 'std'],
    'purchase_day_cos': ['mean', 'max', 'min', 'std'],
    'purchase_hour_sin': ['mean', 'max', 'min', 'std'],
    'purchase_hour_cos': ['mean', 'max', 'min', 'std'],
    'purchase_date': [np.ptp, 'min', 'max'],
    'month_lag': ['mean', 'max', 'min', 'std'],
    'month_diff': ['mean']
    }
    
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    
    return agg_history

In [11]:
def encode(df, col):
    # この方法だと場合によって最大値が変化するデータでは正確な値は出ない
    # 例：月の日数が30日や31日の場合がある
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / df[col].max())
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / df[col].max())
    return df

historical_transactions = encode(historical_transactions, 'purchase_hour')
historical_transactions = encode(historical_transactions, 'purchase_day')
historical_transactions = encode(historical_transactions, 'purchase_month')

authorized_transactions= encode(authorized_transactions, 'purchase_hour')
authorized_transactions = encode(authorized_transactions, 'purchase_day')
authorized_transactions = encode(authorized_transactions, 'purchase_month')

new_transactions= encode(new_transactions, 'purchase_hour')
new_transactions = encode(new_transactions ,'purchase_day')
new_transactions = encode(new_transactions, 'purchase_month')

#historical_transactions.drop(columns=['purchase_hour','purchase_day','purchase_month'])
#authorized_transactions.drop(columns=['purchase_hour','purchase_day','purchase_month'])
#new_transactions.drop(columns=['purchase_hour','purchase_day','purchase_month'])

In [12]:
history = aggregate_transactions(historical_transactions)
history.columns = ['hist_' + c if c != 'card_id' else c for c in history.columns]
history[:5]

,card_id,hist_transactions_count,hist_category_1_sum,hist_category_1_mean,hist_category_2_1.0_mean,hist_category_2_2.0_mean,hist_category_2_3.0_mean,hist_category_2_4.0_mean,hist_category_2_5.0_mean,hist_category_3_A_mean,hist_category_3_B_mean,hist_category_3_C_mean,hist_merchant_id_nunique,hist_merchant_category_id_nunique,hist_state_id_nunique,hist_city_id_nunique,hist_subsector_id_nunique,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,hist_installments_mean,hist_installments_max,hist_installments_min,hist_installments_std,hist_purchase_month_sin_mean,hist_purchase_month_sin_max,hist_purchase_month_sin_min,hist_purchase_month_sin_std,hist_purchase_month_cos_mean,hist_purchase_month_cos_max,hist_purchase_month_cos_min,hist_purchase_month_cos_std,hist_purchase_day_sin_mean,hist_purchase_day_sin_max,hist_purchase_day_sin_min,hist_purchase_day_sin_std,hist_purchase_day_cos_mean,hist_purchase_day_cos_max,hist_purchase_day_cos_min,hist_purchase_day_cos_std,hist_purchase_hour_sin_mean,hist_purchase_hour_sin_max,hist_purchase_hour_sin_min,hist_purchase_hour_sin_std,hist_purchase_hour_cos_mean,hist_purchase_hour_cos_max,hist_purchase_hour_cos_min,hist_purchase_hour_cos_std,hist_purchase_date_ptp,hist_purchase_date_min,hist_purchase_date_max,hist_month_lag_mean,hist_month_lag_max,hist_month_lag_min,hist_month_lag_std,hist_month_diff_mean
0,C_ID_00007093c1,35,4.0,0.114286,0.000000,0.000000,0.885714,0.0,0.000000,0.000000,0.828571,0.171429,11,8,2,3,7,-14.401965,-0.411485,1.507069,-0.728876,0.430059,45,1.285714,3,1,0.667367,0.075256,1.000000,-1.000000e+00,7.133553e-01,-0.174231,1.000000e+00,-1.0,0.695311,0.163046,0.998717,-0.998717,0.727578,-0.227937,1.000000,-0.994869,0.647451,-0.465727,0.942261,-0.997669,0.562076,-0.427338,0.962917,-0.990686,0.549738,28858113.0,1.489250e+09,1.518108e+09,-6.028571,0,-11,3.535415,11.485714
1,C_ID_0001238066,3,0.0,0.000000,0.333333,0.000000,0.000000,0.0,0.666667,0.000000,1.000000,0.000000,2,1,2,2,1,-1.789928,-0.596643,-0.596643,-0.596643,0.000000,3,1.000000,1,1,0.000000,0.288675,0.866025,-2.449294e-16,5.000000e-01,0.833333,1.000000e+00,0.5,0.288675,0.165069,0.897805,-0.201299,0.634567,0.506222,0.979530,-0.440394,0.819794,-0.785523,-0.398401,-0.979084,0.335257,-0.170100,0.203456,-0.917211,0.647018,3609150.0,1.514660e+09,1.518269e+09,-1.333333,0,-2,1.154701,11.000000
2,C_ID_0001506ef0,4,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.750000,0.250000,0.000000,4,4,1,1,4,-2.905782,-0.726445,-0.708590,-0.740491,0.016203,1,0.250000,1,0,0.500000,0.433013,0.866025,-2.449294e-16,5.000000e-01,0.250000,1.000000e+00,-1.0,0.866025,0.604408,0.988468,-0.299363,0.608319,-0.266878,0.347305,-0.954139,0.617444,-0.217900,0.398401,-0.997669,0.577704,-0.741706,-0.068242,-0.990686,0.450310,22098875.0,1.496772e+09,1.518871e+09,-2.500000,0,-8,3.785939,11.500000
3,C_ID_0001793786,27,2.0,0.074074,0.111111,0.296296,0.111111,0.0,0.000000,0.814815,0.185185,0.000000,17,14,4,8,11,-11.825338,-0.437975,1.236592,-0.745405,0.498150,5,0.185185,1,0,0.395847,-0.185185,1.000000,-1.000000e+00,8.565152e-01,-0.350522,6.123234e-17,-1.0,0.376350,0.171487,0.998717,-0.988468,0.724093,-0.101690,0.979530,-0.954139,0.687658,-0.413546,0.398401,-0.979084,0.470132,-0.442236,1.000000,-0.990686,0.660866,16780236.0,1.488636e+09,1.505416e+09,-3.518519,-1,-7,2.375684,15.444444
4,C_ID_000183fdda,7,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.428571,0.571429,4,4,1,1,4,-0.941381,-0.134483,0.334099,-0.714541,0.452741,19,2.714286,6,1,2.288689,-0.866025,-0.866025,-8.660254e-01,4.532467e-17,-0.357143,5.000000e-01,-0.5,0.377964,0.011488,0.988468,-0.998717,0.791276,0.401687,1.000000,-0.440394,0.593423,-0.028917,0.942261,-0.887885,0.780331,-0.105983,1.000000,-0.775711,0.737340,6701589.0,1.502099e+09,1.508801e+09,-5.714286,-4,-6,0.755929,11.285714


In [13]:
authorized = aggregate_transactions(authorized_transactions)
authorized.columns = ['auth_' + c if c != 'card_id' else c for c in authorized.columns]
authorized[:5]

,card_id,auth_transactions_count,auth_category_1_sum,auth_category_1_mean,auth_category_2_1.0_mean,auth_category_2_2.0_mean,auth_category_2_3.0_mean,auth_category_2_4.0_mean,auth_category_2_5.0_mean,auth_category_3_A_mean,auth_category_3_B_mean,auth_category_3_C_mean,auth_merchant_id_nunique,auth_merchant_category_id_nunique,auth_state_id_nunique,auth_city_id_nunique,auth_subsector_id_nunique,auth_purchase_amount_sum,auth_purchase_amount_mean,auth_purchase_amount_max,auth_purchase_amount_min,auth_purchase_amount_std,auth_installments_sum,auth_installments_mean,auth_installments_max,auth_installments_min,auth_installments_std,auth_purchase_month_sin_mean,auth_purchase_month_sin_max,auth_purchase_month_sin_min,auth_purchase_month_sin_std,auth_purchase_month_cos_mean,auth_purchase_month_cos_max,auth_purchase_month_cos_min,auth_purchase_month_cos_std,auth_purchase_day_sin_mean,auth_purchase_day_sin_max,auth_purchase_day_sin_min,auth_purchase_day_sin_std,auth_purchase_day_cos_mean,auth_purchase_day_cos_max,auth_purchase_day_cos_min,auth_purchase_day_cos_std,auth_purchase_hour_sin_mean,auth_purchase_hour_sin_max,auth_purchase_hour_sin_min,auth_purchase_hour_sin_std,auth_purchase_hour_cos_mean,auth_purchase_hour_cos_max,auth_purchase_hour_cos_min,auth_purchase_hour_cos_std,auth_purchase_date_ptp,auth_purchase_date_min,auth_purchase_date_max,auth_month_lag_mean,auth_month_lag_max,auth_month_lag_min,auth_month_lag_std,auth_month_diff_mean
0,C_ID_00007093c1,114,24.0,0.210526,0.000000,0.000000,0.780702,0.0,0.008772,0.0,0.842105,0.157895,28,18,3,4,13,-62.443077,-0.547746,1.206539,-0.728876,0.237426,147,1.289474,6,1,0.795159,-0.029212,1.000000,-1.0,0.692263,-0.058738,1.000000,-1.000000e+00,0.724762,0.257764,0.998717,-0.968077,0.637166,0.155501,1.00000,-0.994869,0.715153,-0.397843,0.979084,-0.997669,0.644185,-0.265163,1.0,-0.990686,0.602712,32627654.0,1.487081e+09,1.519708e+09,-5.798246,0,-12,3.441495,11.570175
1,C_ID_0001238066,120,2.0,0.016667,0.783333,0.000000,0.000000,0.0,0.150000,0.0,0.708333,0.266667,65,29,6,18,17,-70.657272,-0.588811,0.768095,-0.734887,0.192614,195,1.625000,10,-1,1.501050,0.082436,0.866025,-1.0,0.571997,0.786960,1.000000,-1.836970e-16,0.223291,-0.177791,0.998717,-0.998717,0.732300,-0.049337,0.97953,-0.994869,0.661672,-0.388772,0.979084,-0.997669,0.563825,-0.017187,1.0,-0.990686,0.733344,13110825.0,1.506638e+09,1.519748e+09,-1.825000,0,-5,1.294218,11.266667
2,C_ID_0001506ef0,62,0.0,0.000000,0.032258,0.000000,0.967742,0.0,0.000000,1.0,0.000000,0.000000,28,19,2,3,12,-31.696098,-0.511227,1.493545,-0.740491,0.484575,0,0.000000,0,0,0.000000,0.160711,1.000000,-1.0,0.622642,0.504167,1.000000,-1.000000e+00,0.586607,0.399532,0.998717,-0.998717,0.682862,-0.036870,1.00000,-0.994869,0.621666,-0.247064,0.942261,-0.997669,0.536369,-0.689431,1.0,-0.990686,0.428431,34460275.0,1.484411e+09,1.518871e+09,-4.983871,0,-13,4.248402,11.612903
3,C_ID_0001793786,189,0.0,0.000000,0.042328,0.359788,0.063492,0.0,0.000000,1.0,0.000000,0.000000,114,45,4,9,22,-24.960674,-0.132067,4.554145,-0.737892,0.867916,0,0.000000,0,0,0.000000,-0.233272,1.000000,-1.0,0.729199,-0.388268,0.866025,-1.000000e+00,0.517039,-0.054534,0.998717,-0.998717,0.690566,-0.119602,1.00000,-0.994869,0.714889,-0.503148,0.997669,-0.997669,0.491637,-0.121833,1.0,-0.990686,0.702983,24487497.0,1.484994e+09,1.509481e+09,-3.301587,0,-9,2.301491,15.349206
4,C_ID_000183fdda,137,4.0,0.029197,0.051095,0.007299,0.905109,0.0,0.007299,0.0,0.729927,0.240876,71,34,7,9,20,-67.896553,-0.495595,2.764788,-0.737892,0.523760,245,1.788321,10,-1,2.108912,-0.065955,0.866025,-1.0,0.715655,0.613452,1.000000,-1.836970e-16,0.334243,0.199701,0.998717,-0.998717,0.693076,0.022043,1.00000,-0.994869,0.697378,-0.460366,0.816970,-0.997669,0.424229,0.135026,1.0,-0.990686,0.771700,15148616.0,1.504444e+09,1.519592e+09,-2.284672,0,-5,1.782055,11.445255


In [14]:
new = aggregate_transactions(new_transactions)
new.columns = ['new_' + c if c != 'card_id' else c for c in new.columns]
new[:5]

,card_id,new_transactions_count,new_category_1_sum,new_category_1_mean,new_category_2_1.0_mean,new_category_2_2.0_mean,new_category_2_3.0_mean,new_category_2_4.0_mean,new_category_2_5.0_mean,new_category_3_A_mean,new_category_3_B_mean,new_category_3_C_mean,new_merchant_id_nunique,new_merchant_category_id_nunique,new_state_id_nunique,new_city_id_nunique,new_subsector_id_nunique,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_installments_sum,new_installments_mean,new_installments_max,new_installments_min,new_installments_std,new_purchase_month_sin_mean,new_purchase_month_sin_max,new_purchase_month_sin_min,new_purchase_month_sin_std,new_purchase_month_cos_mean,new_purchase_month_cos_max,new_purchase_month_cos_min,new_purchase_month_cos_std,new_purchase_day_sin_mean,new_purchase_day_sin_max,new_purchase_day_sin_min,new_purchase_day_sin_std,new_purchase_day_cos_mean,new_purchase_day_cos_max,new_purchase_day_cos_min,new_purchase_day_cos_std,new_purchase_hour_sin_mean,new_purchase_hour_sin_max,new_purchase_hour_sin_min,new_purchase_hour_sin_std,new_purchase_hour_cos_mean,new_purchase_hour_cos_max,new_purchase_hour_cos_min,new_purchase_hour_cos_std,new_purchase_date_ptp,new_purchase_date_min,new_purchase_date_max,new_month_lag_mean,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_month_diff_mean
0,C_ID_00007093c1,2,0,0.000000,0.500000,0.000000,0.50000,0.0,0.000000,0.0,1.000000,0.000000,2,2,2,2,2,-1.328125,-0.664062,-0.656738,-0.671875,0.010704,2,1.000000,1,1,0.000000,0.866025,8.660254e-01,0.866025,0.000000,-5.000000e-01,-5.000000e-01,-5.000000e-01,0.000000,0.769673,0.968077,0.571268,0.280586,0.285055,0.820763,-0.250653,0.757606,-0.403047,0.136167,-0.942261,0.762563,-0.662783,-0.334880,-0.990686,0.463725,537024.0,1.522754e+09,1.523291e+09,2.000000,2,2,0.000000,12.000000
1,C_ID_0001238066,26,2,0.076923,0.769231,0.000000,0.00000,0.0,0.115385,0.0,0.807692,0.153846,25,15,4,8,9,-14.851562,-0.571289,-0.078308,-0.740723,0.173462,42,1.615385,10,-1,2.079941,0.953624,1.000000e+00,0.866025,0.065000,-1.730769e-01,6.123234e-17,-5.000000e-01,0.242582,-0.242342,0.897805,-0.998717,0.604566,-0.005315,0.979530,-0.994869,0.783193,-0.426432,0.631088,-0.997669,0.531247,0.021570,1.000000,-0.990686,0.753773,5195343.0,1.519923e+09,1.525118e+09,1.346154,2,1,0.485165,11.192308
2,C_ID_0001506ef0,2,0,0.000000,0.000000,0.000000,1.00000,0.0,0.000000,1.0,0.000000,0.000000,1,2,1,1,2,-1.447266,-0.723633,-0.715332,-0.731934,0.011742,0,0.000000,0,0,0.000000,1.000000,1.000000e+00,1.000000,0.000000,6.123234e-17,6.123234e-17,6.123234e-17,0.000000,-0.534623,-0.101168,-0.968077,0.612997,-0.622761,-0.250653,-0.994869,0.526241,0.180646,0.631088,-0.269797,0.637022,0.093603,0.962917,-0.775711,1.229396,471152.0,1.521239e+09,1.521710e+09,1.000000,1,1,0.000000,11.000000
3,C_ID_0001793786,31,0,0.000000,0.483871,0.258065,0.16129,0.0,0.032258,1.0,0.000000,0.000000,31,21,5,7,14,-0.229004,-0.007385,3.130859,-0.737793,0.947266,0,0.000000,0,0,0.000000,-0.338710,-2.449294e-16,-0.500000,0.237595,9.092430e-01,1.000000e+00,8.660254e-01,0.063664,-0.423101,0.897805,-0.988468,0.479171,0.178306,1.000000,-0.994869,0.765437,-0.462016,0.519584,-0.997669,0.453787,0.100062,1.000000,-0.990686,0.772328,3981096.0,1.510761e+09,1.514742e+09,1.322581,2,1,0.475191,15.032258
4,C_ID_000183fdda,11,0,0.000000,0.000000,0.000000,1.00000,0.0,0.000000,0.0,0.545455,0.363636,11,9,2,2,6,-6.589844,-0.599121,-0.107666,-0.732422,0.182861,16,1.454545,4,-1,1.293340,0.963461,1.000000e+00,0.866025,0.062579,-1.363636e-01,6.123234e-17,-5.000000e-01,0.233550,0.333803,0.998717,-0.848644,0.672689,0.407510,0.979530,-0.758758,0.585023,-0.562826,0.136167,-0.997669,0.384317,-0.352479,0.962917,-0.990686,0.683509,5106807.0,1.519994e+09,1.525100e+09,1.272727,2,1,0.467099,11.636364


The second function first aggregates on the two variables `card_id` and `month_lag`. Then a second grouping is performed to aggregate over time:

In [15]:
def aggregate_per_month(history):
    grouped = history.groupby(['card_id', 'month_lag'])

    agg_func = {
            'purchase_amount': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            'installments': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            }

    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = ['_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)

    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = ['_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True)
    
    return final_group
#___________________________________________________________
final_group =  aggregate_per_month(authorized_transactions) 
final_group[:10]

,card_id,month_lag_mean,month_lag_std,purchase_amount_count_mean,purchase_amount_count_std,purchase_amount_sum_mean,purchase_amount_sum_std,purchase_amount_mean_mean,purchase_amount_mean_std,purchase_amount_min_mean,purchase_amount_min_std,purchase_amount_max_mean,purchase_amount_max_std,purchase_amount_std_mean,purchase_amount_std_std,installments_count_mean,installments_count_std,installments_sum_mean,installments_sum_std,installments_mean_mean,installments_mean_std,installments_min_mean,installments_min_std,installments_max_mean,installments_max_std,installments_std_mean,installments_std_std
0,C_ID_00007093c1,-6.000000,3.894440,8.769231,3.539158,-4.803313,2.175011,-0.536131,0.075418,-0.702649,0.041293,-0.126683,0.446167,0.197339,0.120199,8.769231,3.539158,11.307692,4.210792,1.369646,0.423190,1.000000,0.000000,2.538462,1.613246,0.625428,0.675607
1,C_ID_0001238066,-2.500000,1.870829,20.000000,11.696153,-11.776212,6.676914,-0.603215,0.046045,-0.730351,0.002699,-0.160977,0.485869,0.151393,0.093832,20.000000,11.696153,32.500000,24.501020,1.652873,0.410539,0.666667,0.816497,4.666667,2.875181,1.256525,0.521292
2,C_ID_0001506ef0,-6.230769,4.225988,4.769231,4.342692,-2.438161,2.902502,-0.446660,0.271125,-0.725724,0.011212,0.161333,0.748701,0.496308,0.415117,4.769231,4.342692,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,C_ID_0001793786,-4.500000,3.027650,18.900000,12.160501,-2.496067,4.018810,-0.005687,0.271221,-0.646277,0.166230,2.190924,1.290775,0.880276,0.259048,18.900000,12.160501,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,C_ID_000183fdda,-2.500000,1.870829,22.833333,7.167054,-11.316092,4.800612,-0.489227,0.141015,-0.728405,0.009876,1.151797,1.062785,0.472149,0.315407,22.833333,7.167054,40.833333,14.048725,1.834943,0.362557,0.333333,1.032796,9.166667,2.041241,2.144389,0.665529
5,C_ID_00024e244b,-6.384615,4.330867,4.076923,4.290717,-2.557395,2.089708,-0.685118,0.071894,-0.728885,0.016051,-0.425292,0.813979,0.108521,0.208719,4.076923,4.290717,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,C_ID_0002709b5a,-5.000000,3.316625,6.090909,3.505839,-3.994622,2.440540,-0.638637,0.060836,-0.700104,0.030670,-0.546063,0.092512,0.076295,0.051724,6.090909,3.505839,10.727273,8.427229,1.965729,1.287108,0.818182,0.603023,3.818182,3.341203,1.600058,1.734256
7,C_ID_00027503e2,-4.000000,2.738613,2.666667,1.224745,-1.974976,0.909524,-0.740077,0.002424,-0.742206,0.003025,-0.737762,0.002652,0.003165,0.001086,2.666667,1.224745,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,C_ID_000298032a,-5.250000,4.334249,3.500000,1.772811,-1.844563,0.699127,-0.590214,0.168008,-0.686623,0.020383,-0.210304,1.127668,0.213951,0.442601,3.500000,1.772811,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,C_ID_0002ba3c2e,-4.666667,3.141125,9.166667,4.665476,-5.843051,3.021165,-0.638037,0.044976,-0.721448,0.026047,-0.387276,0.222295,0.107390,0.060790,9.166667,4.665476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
def successive_aggregates(df, field1, field2):
    t = df.groupby(['card_id', field1])[field2].mean()
    u = pd.DataFrame(t).reset_index().groupby('card_id')[field2].agg(['mean', 'min', 'max', 'std'])
    u.columns = [field1 + '_' + field2 + '_' + col for col in u.columns.values]
    u.reset_index(inplace=True)
    return u

In [17]:
additional_fields = successive_aggregates(new_transactions, 'category_1', 'purchase_amount')
additional_fields = additional_fields.merge(successive_aggregates(new_transactions, 'installments', 'purchase_amount'),
                                            on = 'card_id', how='left')
additional_fields = additional_fields.merge(successive_aggregates(new_transactions, 'city_id', 'purchase_amount'),
                                            on = 'card_id', how='left')
additional_fields = additional_fields.merge(successive_aggregates(new_transactions, 'category_1', 'installments'),
                                            on = 'card_id', how='left')


<a id="3"></a> <br>
## 3. Training the model
We now train the model with the features we previously defined. A first step consists in merging all the dataframes:

In [18]:
train = pd.merge(train, history, on='card_id', how='left')
test = pd.merge(test, history, on='card_id', how='left')

train = pd.merge(train, authorized, on='card_id', how='left')
test = pd.merge(test, authorized, on='card_id', how='left')

train = pd.merge(train, new, on='card_id', how='left')
test = pd.merge(test, new, on='card_id', how='left')

train = pd.merge(train, final_group, on='card_id', how='left')
test = pd.merge(test, final_group, on='card_id', how='left')

train = pd.merge(train, auth_mean, on='card_id', how='left')
test = pd.merge(test, auth_mean, on='card_id', how='left')

train = pd.merge(train, additional_fields, on='card_id', how='left')
test = pd.merge(test, additional_fields, on='card_id', how='left')

In [19]:
# cols = [c for c in train if c.startswith('hist')]
# train.loc[train['hist_transactions_count'].isnull(), cols] = 0
# test.loc[test['hist_transactions_count'].isnull(), cols] = 0

# cols = [c for c in train if c.startswith('new')]
# train.loc[train['new_transactions_count'].isnull(), cols] = 0
# test.loc[test['new_transactions_count'].isnull(), cols] = 0

In [20]:
# cols = [c for c in train if c.endswith('std')]
# for c in cols:
#     train.loc[train[c].isnull(), c] = 0
#     test.loc[test[c].isnull(), c] = 0

In [21]:
# train['transactions_ratio'] = train['new_transactions_count'] / train['hist_transactions_count']
# test['transactions_ratio'] = test['new_transactions_count'] / test['hist_transactions_count']

In [22]:
# hist_columns = [(c, c.replace('new', 'auth')) for c in train.columns if 'hist' in c]
# for c in hist_columns:
#     col_name = 'ratio_{}_{}'.format(c[0], c[1])
#     train[col_name] = train[c[0]] / train[c[1]]
#     test[col_name] = test[c[0]] / test[c[1]]

In [23]:
test.to_csv('../../../input/test_1.csv')
train.to_csv('../../../input/train_1.csv')

and to define the features we want to keep to train the model. For that purpose, I use the results obtained in the [Selecting features kernel](https://www.kaggle.com/fabiendaniel/selecting-features/notebook):